# Visualize Isobaric Correction Point

Notebook to plot isobaric corrections of selected channels for 1 point, a.k.a. FoV.
It also plots the spectrum in TOF bin units and in mass units.
In this script 2 images are plotted in the same axis (with and without isobaric corrections applied), allowing the user to toggle the visibility between both of them: once the notebook has run, select the desired plot and press 't' to toggle the visibility of both plots. This is useful in order to visualize the change in a channel before and afer isobaric corrections.

Accepted format for the images: IONpath's **MIBItiff** as from the MIBI/O software or the MIBItracker.

Please update the parameters in the **input parameters** section before running the notebook.

# load headers

In [ ]:
# magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

# input parameters

In [ ]:
data_path = os.path.expanduser('~/common/path/to/data')

slide_num = 0
point_num = 0
run_name = 'run_name_for_slide0'

# load tiff (with isob corrs applied)
tiff_file = os.path.join(data_path,
                         f'raw_msdf/{run_name}/{run_name}_TIFF',
                         f'sl{slide_num:02}_pt{point_num:02}/bg_au_050_ta_020/',
                         f'Point{point_num}_RowNumber0_Depth_Profile0-MassCorrected.tiff')

# load bg tiff (bg here means: before isob corrs)
tiff_file_bg = os.path.join(data_path,
                            f'raw_msdf/{run_name}/{run_name}_TIFF',
                            f'sl{slide_num:02}_pt{point_num:02}/bg_au_050_ta_020/',
                            f'Point{point_num}_RowNumber0_Depth_Profile0.tiff')

# load isobaric corrections json file
isob_corr_file = os.path.join(data_path,
                              f'raw_msdf/{run_name}/{run_name}_TIFF',
                              f'sl{slide_num:02}_pt{point_num:02}/bg_au_050_ta_020/',
                              f'Point{point_num}_RowNumber0_Depth_Profile0-MassCorrected.json')

# load spectrum csv file
spectrum_file = os.path.join(data_path,
                             f'raw_msdf/{run_name}/Point{point_num}/RowNumber0/Depth_Profile0/',
                             'AccumulatedSpectrum.csv')
# load panel
panel_file = os.path.join(data_path,
                          'panel',
                          'Panel.csv')

# select channels
a_masses = np.array([23,  39,  89, 113, 115, 146, 148, 149, 159, 181, 197])
a_masses.sort()
print(f'a_masses = {a_masses}')

#anonymize_targets = True
anonymize_targets = False

show_isobaric_donors = True
#show_isobaric_donors = False

# read isobaric corrections

In [ ]:
# dictionary of corrections: keys are masses of recipients, values are lists of masses of donors to that
# recipient:
# {recipient1: [donor11, donor12, ...], recipient2: [donor21, donor22, ...], ...}
dict_masses = viz.read_isobaric_corrections(isob_corr_file, a_masses)
print(f'dict_masses = {dict_masses}')

# read panel

In [ ]:
panel_df = viz.read_panel_from_csv(panel_file, anonymize_targets=anonymize_targets)
#display(panel_df)

# transform masses to targets for selected channels
dict_channels = {}
for mass, mass_donors in dict_masses.items():
    target = panel_df['Target'][panel_df['Mass'] == mass].values[0]
    l_donors = []
    for mass_donor in mass_donors:
        l_donors.append(panel_df['Target'][panel_df['Mass'] == mass_donor].values[0])
    dict_channels[mass] = {'target': target, 'donors': l_donors, 'mass_donors': mass_donors}
#print(f'dict_channels = {dict_channels}')
df_channels = pd.DataFrame.from_dict(dict_channels, orient='index')
df_channels.index.name = 'mass'
if show_isobaric_donors:
    display(df_channels)
else:
    display(df_channels[['target']])

# plot spectrum

In [ ]:
# read spectrum csv file
spectrum_df = viz.read_spectrum_from_csv(spectrum_file)
#display(spectrum_df)

# light plot style
plt.style.use('default')

spec_axes = []

# plot TOF bin spectrum
spec_axes.append(viz.plot_spectrum(spectrum_df, plot_mass=False, plot_wide=True,
                                   ax=None,
                                   title='TOF bin spectrum'))

# plot mass spectrum
spec_axes.append(viz.plot_spectrum(spectrum_df, plot_mass=True, plot_wide=True,
                                   ax=None,
                                   title='mass spectrum'))

# plot vertical lines for know masses
df_known_masses = viz.get_know_masses_df()
for _, row in df_known_masses.iterrows():
    #print(f'Element {row["Element"]}, mass {row["Mass"]}')
    spec_axes[1].axvline(x=row['Mass'], color='gray', linestyle=':', linewidth=1)
#display(df_known_masses)

# plot vertical lines for selected channels
for mass in df_channels.index.values:
    spec_axes[1].axvline(x=mass, color='black', linestyle='--', linewidth=1)

    # plot vertical lines for donors
    if show_isobaric_donors:
        mass_donors = df_channels.loc[mass]['mass_donors']
        for mass_donor in mass_donors:
            spec_axes[1].axvline(x=mass_donor, color='red', linestyle='--', linewidth=1)

# plot images

In [ ]:
# read MIBItiff image file
image = tiff.read(tiff_file)
if not anonymize_targets:
    #print(f'metadata {image.metadata}')
    #print(f'channels {image.channels}') # sorted by target name
    print(f'channels {sorted(image.channels)}') # sorted by mass

# read MIBItiff bg image file
image_bg = tiff.read(tiff_file_bg)

# dark plot style
plt.style.use('dark_background')

# loop over channels
# im_axes = [] # not used for now
for ch_order, channel in enumerate(sorted(image.channels)):
    mass = channel[0]
    target = channel[1]
    #print() # debug
    #print(f'channel {ch_order}: {channel}, mass {mass}, target {target}') # debug
    #if mass > 70 and mass < 180 and mass != 128: # all biological channels
    if mass in df_channels.index.values: # selected channels
        print()
        if show_isobaric_donors:
            try:
                mass_donors = df_channels.loc[mass]['mass_donors']
                donors = df_channels.loc[mass]['donors'] # this should be anonymized already
            except KeyError:
                print(f'INFO: no donors found for channel {mass}')
                mass_donors = []
                donors = []                
        else:
            mass_donors = []
            donors = []
        if anonymize_targets:
            try:
                target = df_channels.loc[mass]['target']
                # no need to anonymize donors
            except:
                target = 'dummy'
            finally:
                print('WARNING! TIFF files and \'channel\' variable might not be anonymized!')
#         else:
#             print(f'channel {ch_order}: {channel}, mass {mass}, target {target}, donors ({mass_donors}, {donors})') # debug (channel var not anonymized!)
        if show_isobaric_donors:
            print(f'channel {ch_order}: mass {mass}, target {target}, donors ({mass_donors}, {donors})')
        else:
            print(f'channel {ch_order}: mass {mass}, target {target}')
        im = image[mass]
        im_bg = image_bg[mass]
        counts = im.sum()
        n_plots = 1 + len(mass_donors)
        fig_size = 8 # inch
        #fig_size = 5 # inch # debug
        fig, ax = plt.subplots(1, n_plots, figsize=(n_plots*fig_size, fig_size))
        plt.ioff() # deactivate interactive mode in order not to show the next plot and avoid empty white space in the jupyter notebook (probably not necessary for pure python code)
        try:
            ax_recipient = ax[0]
        except:
            # in case the channel has no donors
            ax_recipient = ax
        viz.plot_toggle_image(im, im_bg,
                              ax=ax_recipient,
                              #title=str(channel) + ' ' + str("{:.2e}".format(counts)), # debug (not anonymized)
                              title=f'({mass}, \'{target}\')' + ' ' + str("{:.2e}".format(counts)),
                              #brighten_image=True, hi_res=True)
                              brighten_image=True, hi_res=False)
        plt.ioff() # deactivate interactive mode in order not to show the next plot and avoid empty white space in the jupyter notebook (probably not necessary for pure python code)
        for n_donor, (mass_donor, donor) in enumerate(zip(mass_donors, donors)):
            im_donor = image[mass_donor]
            viz.plot_image(im_donor,
                                  ax=ax[1 + n_donor],
                                  #title=str(channel) + ' ' + str("{:.2e}".format(counts)), # debug (not anonymized)
                                  title=f'({mass_donor}, \'{donor}\')' + ' ' + str("{:.2e}".format(counts)),
                                  #brighten_image=True, hi_res=True)
                                  brighten_image=True, hi_res=False)
        plt.ion() # turn interactive mode back on to be able to toggle
        fig.tight_layout()
        #break # debug (show only 1 plot)